In [1]:
import numpy as np
import json

In [2]:
# config = {
#     "age_starting": 24,
#     "age_retirement": 60,
#     "age_sim_end": 100,
#     "number_of_runs": 10000,
#     "amount_starting": 0,
#     "contribution_monthly": 3500,
#     "retirement_drawdown_monthly": 10000,
#     "inflation_mean": 0.043,
#     "inflation_std":  0.005,
#     "growth_mean": 0.051,
#     "growth_std": 0.096,
# }

In [3]:
config = json.load(open('config.json'))['person1']

In [4]:
config['growth_std'] = config['growth_std']/np.sqrt(12)  # fact sheets shows annulaised standard devivation...

In [5]:
periods = (config['age_sim_end'] - config['age_starting'])*12 + 1
drawdown_start_period = (config['age_retirement'] - config['age_starting'])*12 + 1

In [6]:
size = (periods, config['number_of_runs'])

In [7]:
inflation = np.random.normal(config['inflation_mean'], config['inflation_std'], size=size)
growth = np.random.normal(config['growth_mean'], config['growth_std'], size=size)
delta_growth = (growth - inflation)/12

In [8]:
scenarios = np.ones(size)

In [9]:
scenarios[0,:] = config['amount_starting']
for p in range(periods)[1:]:
    if drawdown_start_period > p:
        scenarios[p,:] = (scenarios[p-1,:] + config['contribution_monthly']) * (1+delta_growth[p,:])
    else:
        scenarios[p,:] = (scenarios[p-1,:] - config['retirement_drawdown_monthly']) * (1+delta_growth[p,:])
        
scenarios[scenarios < 0] = 0

In [10]:
percentiles = [1,10,25,50,75,90,99]
scenario_stats = np.percentile(scenarios, percentiles, axis=1)
scenario_stats.shape

(7, 841)

In [11]:
from bokeh.plotting import figure, show
from bokeh.palettes import Spectral as palette
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [12]:
# prepare some data
x = config['age_starting'] + np.arange(periods)/12

# create a new plot with a title and axis labels
plot = figure(title="Possible outcomes", x_axis_label="age", y_axis_label="currency", plot_width=950, plot_height=500)

colors = list(palette[len(percentiles)])
colors[len(colors)//2] = '#000000'

for n, p in enumerate(percentiles): 
    y = scenario_stats[n,:]
    plot.line(x, y, legend_label=f"{p}", line_color=colors[len(percentiles) -1 - n], line_width=2)

# show the results
show(plot)